In [ ]:
import pandas as pd
import numpy as np
import difflib
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)

In [ ]:
df = pd.read_csv('../data/gdp.csv', header=1)

In [ ]:
df.head()

In [ ]:
df.rename(columns={'Time': 'Country'}, inplace=True)
df.drop(0, inplace=True)
df.head()

In [ ]:
import difflib
countries = pd.read_csv('../data/country-codes_csv.csv')
def get_country_iso(name):
    p = difflib.get_close_matches(name, countries['official_name_en'].dropna().unique(), n=1, cutoff=0.5)
    if len(p)>0:
        return countries[countries['official_name_en']==p[0]]['ISO3166-1-Alpha-3'].to_list()[0]
    else:
        return ""

In [ ]:
df['country_iso'] = [get_country_iso(x) for x in df['Country']]
df[df['country_iso']=='']

In [ ]:
df['country_val'] = [countries[countries['ISO3166-1-Alpha-3']==x]['official_name_en'].to_list()[0] for x in df['country_iso']]

In [ ]:
df[['Country', 'country_iso', 'country_val']]

Line 44, North Macedonia, was incorrectly assigned as New Caledonia. Fixing it to the correct ISO code:

In [ ]:
df.loc[153,'country_iso'] = 'MKD'

In [ ]:
df.drop(columns=['Country', 'country_val'], inplace=True)

In [ ]:
df.columns

In [ ]:
df

In [ ]:
df_new = df.melt(id_vars=["country_iso"], var_name="year", value_name="gdp_per_capta_usd")

In [ ]:
pd.set_option('display.max_rows', None)
df_new = df_new.sort_values(['country_iso', 'year'])
df_new.replace('..', np.nan, inplace=True)
df_new.dropna(inplace=True)
df_new

In [ ]:
df_new.to_csv('../data/converted/gdp_per_capta.csv', index=False)

In [ ]:
df_saved = pd.read_csv('../data/converted/gdp_per_capta.csv')
df_saved